In [12]:
import pandas as pd
import numpy as np
import sys
sys.path.append('/Users/sva21/Documents/mangrove/code/backtester-RL/')

## Orders

In [13]:
import src.order

In [14]:
order_type = 'bid'

qty_b1 = 100
price_b1 = 99
bid1 = src.order.Order(order_type, qty_b1, price_b1)

qty_b2 = 10
price_b2 = 98
bid2 = src.order.Order(order_type, qty_b2, price_b2)

order_type = 'ask'

qty_a1 = 100
price_a1 = 101
ask1 = src.order.Order(order_type, qty_a1, price_a1)

qty_a2 = 10
price_a2 = 102
ask2 = src.order.Order(order_type, qty_a2, price_a2)


## Order Book

In [ ]:
from src.order_book import OrderBook
orderbook = OrderBook([bid1, bid2], [ask1, ask2])
print(orderbook)

## Main Script


In [16]:
from src.kandel import kandel_simulator
from src.time_series import load_csv
import pandas as pd
from src.time_series import get_timedelta_unit


In [17]:
ts = load_csv('data/ETHUSDC-1s-2024-08-clean.csv')

In [ ]:
window = 3600*12
quote = 75000
base = 0
vol_mult = 1
n_points = 10
step_size = 1
days = 3

order_book = OrderBook()
transactions, res, order_book = kandel_simulator(ts = ts[-days*24*3600:], 
            quote = quote,
            base = base, 
            vol_mult=vol_mult,
            n_points= n_points, 
            step_size=step_size,
            order_book= order_book,
            window= window)
res.to_pandas().to_csv('results/simul_results.csv')